In [24]:
from langchain.tools import tool
from langchain_openai import ChatOpenAI
import requests
from langchain_core.tools import  InjectedToolArg
from typing import Annotated
from langchain_core.messages import HumanMessage

In [4]:
llm = ChatOpenAI(model="gpt-5-nano")

In [22]:
@tool
def currency_rate(from_currency: str, to_currency: str) -> float:
    """Convert amount from one currency to another using an external API."""
    url = f"https://api.exchangerate-api.com/v4/latest/{from_currency}"
    response = requests.get(url)
    data = response.json()
    rate = data['rates'].get(to_currency)
    if rate:
        return rate
    else:
        raise ValueError("Invalid target currency")

In [20]:
@tool
def multiply(amount: float, rate: Annotated[float, InjectedToolArg]) -> float:
    """Multiply amount by the conversion rate."""
    return amount * rate

In [21]:
# currency_rate("USD", "INR")

In [33]:
messages = [HumanMessage(content="What is the date of USD to INR, and convert 1000 USD to INR?")]

In [34]:
llm_with_tools = llm.bind_tools([currency_rate, multiply])

In [35]:
result = llm_with_tools.invoke(messages)

In [37]:
result.tool_calls

[{'name': 'currency_rate',
  'args': {'from_currency': 'USD', 'to_currency': 'INR'},
  'id': 'call_dQFN9AYZEbvc4OONKnvagYQ0',
  'type': 'tool_call'}]